In [18]:
import os
data_dir = ['E:/PythonModels/archive/data'+i+'/data'+i for i in ['X', 'Y']]
image_paths = [i+'/CameraRGB' for i in data_dir]
seg_paths = [i+'/CameraSeg' for i in data_dir]
images,masks = [],[]
for i in image_paths:
    imgs = os.listdir(i)
    images.extend([i+'/'+img for img in imgs])
for i in seg_paths:
    masks_name = os.listdir(i)
    masks.extend([i+'/'+mask for mask in masks_name])


In [19]:
import albumentations as A

augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.5),
    A.Resize(256,256),
    A.RandomCrop(height=256, width=256, p=0.5)
])

In [20]:
from PIL import Image
import cv2
import numpy as np
import os

# Load your image and segmentation mask file paths

# Define the output directories for augmented images and masks
output_image_dir = "E:\Licenta_DOC\API_Segmentation\data\generated\img_dir"
output_mask_dir = "E:\Licenta_DOC\API_Segmentation\data\generated\\ann_dir"

# Set the number of augmented images you want to generate for each original image
num_augmentations = 5

# Apply augmentation pipeline to each image and mask pair
for img_path, mask_path in zip(images, masks):
    img = np.array(Image.open(img_path))
    mask = np.array(Image.open(mask_path))

    for i in range(num_augmentations):
        augmented = augmentation_pipeline(image=img, mask=mask)
        augmented_image = augmented['image']
        augmented_mask = augmented['mask']

        # Save the augmented image and mask
        img_basename = os.path.basename(img_path)
        mask_basename = os.path.basename(mask_path)

        img_output_path = os.path.join(output_image_dir, f"{img_basename[:-4]}_aug_{i}.jpg")
        mask_output_path = os.path.join(output_mask_dir, f"{mask_basename[:-4]}_aug_{i}.png")

        image_pil = Image.fromarray(augmented_image)
        mask_pil = Image.fromarray(augmented_mask)
        image_pil.save(img_output_path)
        mask_pil.save(mask_output_path)
